In [1]:
from  lxml import etree
import requests
import time
import re
import pandas as pd
import random

In [3]:
lst = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0) Gecko/20100101 Firefox/6.0',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET \
    CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; InfoPath.3)'
]
headers = {'User-Agent': random.choice(lst)}


In [4]:
def get_web_data(dom):
    names = dom.xpath('//span[@class="comment-info"]/a/text()')
    ratings = dom.xpath('//span[@class="comment-info"]/span[2]/@class')
    times = dom.xpath('//span[@class="comment-info"]/span[@class="comment-time "]/@title')
    message = dom.xpath('//div[@class="comment"]//span[@class="short"]/text()')
    user_url= dom.xpath('//span[@class="comment-info"]/a/@href')
    votes = dom.xpath('//div[@class="comment"]//span[@class="votes vote-count"]/text()')
    
    cities = []
    load_times = []
    j = 0
    for i in user_url:
        print('正在爬取第%d条'%j)
        j +=1 
        web_data = requests.get(i, headers=headers)
        dom_url = etree.HTML(web_data.text, etree.HTMLParser(encoding='utf-8'))
        #为了保险起见，可以把div的上级div也写上
        address = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/a/text()')
        load_time = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/div[@class="pl"]/text()')
        cities.append(address)
        load_times.append(load_time)
        time.sleep(random.randint(2,4))
    print(ratings)
    print(len(ratings))
    ratings = ['' if 'rating' not in i else int(re.findall('\d{2}', i)[0]) for i in ratings]
    load_times = ['' if i== [] else i[1].strip()[:-2] for i in load_times]
    cities = ['' if i == [] else i[0] for i in cities]
    data = pd.DataFrame({
        '用户名': names,
        '居住城市': cities,
        '加入时间': load_times,
        '评分': ratings,
        '发表时间': times,
        '短评正文': message,
        '赞同数量': votes
    })
    return data

In [6]:
all_data = pd.DataFrame()
for pg in range(0,480,20):
    url = 'https://movie.douban.com/subject/1889243/comments?start={}&limit=20&status=P&sort=new_score'.format(pg)
    res = requests.get(url, headers=headers)
    dom = etree.HTML(res.text)
    data = get_web_data(dom)
    all_data = pd.concat([all_data, data],axis=0)
all_data

正在爬取第0条
正在爬取第1条
正在爬取第2条
正在爬取第3条
正在爬取第4条
正在爬取第5条
正在爬取第6条
正在爬取第7条
正在爬取第8条
正在爬取第9条
正在爬取第10条
正在爬取第11条
正在爬取第12条
正在爬取第13条
正在爬取第14条
正在爬取第15条
正在爬取第16条
正在爬取第17条
正在爬取第18条
正在爬取第19条
['allstar40 rating', 'allstar50 rating', 'allstar50 rating', 'allstar20 rating', 'allstar30 rating', 'allstar10 rating', 'allstar40 rating', 'allstar40 rating', 'allstar50 rating', 'allstar40 rating', 'allstar50 rating', 'allstar30 rating', 'allstar40 rating', 'allstar30 rating', 'allstar30 rating', 'allstar40 rating', 'allstar30 rating', 'allstar50 rating', 'allstar50 rating', 'allstar30 rating']
20
正在爬取第0条
正在爬取第1条
正在爬取第2条
正在爬取第3条
正在爬取第4条
正在爬取第5条
正在爬取第6条
正在爬取第7条
正在爬取第8条
正在爬取第9条
正在爬取第10条
正在爬取第11条
正在爬取第12条
正在爬取第13条
正在爬取第14条
正在爬取第15条
正在爬取第16条
正在爬取第17条
正在爬取第18条
正在爬取第19条
['allstar30 rating', 'allstar30 rating', 'allstar40 rating', 'allstar30 rating', 'allstar30 rating', 'allstar50 rating', 'allstar50 rating', 'allstar30 rating', 'allstar20 rating', 'allstar50 rating', 'allstar50 rating', 'allstar40 rating', 'allstar40 rati

,用户名,居住城市,加入时间,评分,发表时间,短评正文,赞同数量
0,王祉乔,上海,2017-01-13,40,2022-02-01 11:19:11,1950年 是新中国成立后的第一个虎年,6590
1,雾散,,,50,2022-02-01 10:56:25,“第七穿插连应到一百五十七人，实到一人。”,7579
2,李六岁的娃娃亲,北京,2012-09-19,50,2022-02-01 11:14:14,七连全军就义只剩一面旗，万里能留下来全因为是大年初一。,3959
3,ERic,,,20,2022-02-02 10:56:00,不煽情会更好,1741
4,缅怀,,,30,2022-02-01 11:25:55,吴京终于以主角光环➕360度慢镜头全方位无死角的方式展示了他的牺牲,7938
...,...,...,...,...,...,...,...
15,溯溯,,,40,2022-02-01 17:10:00,感觉还行，但是再出我不会看了,201
16,Panda561,北京,2011-02-26,30,2022-02-01 14:42:44,看了《冬与狮》，就明白七连的结局了,91
17,crystal magic,,,50,2022-02-19 15:29:54,节奏比第一部要好太多了 没有一点冷场，从头打到尾 徐克风格体现淋漓尽致，太压抑了，比第一部...,35
18,Vzun,,,20,2022-02-10 17:51:39,整体观感，这已经不是电影了，，,95
